# instalación

In [9]:
import pandas as pd
from pathlib import Path

# === Rutas fijas (hardcodeadas) ===
ruta_acta =  Path(r"20251003/20251003/ADP_DTM_FACT.Acta.csv")
ruta_insumo =  Path(r"20251003/20251003/ADP_DTM_DIM.Insumo.csv")
ruta_proyeccion =  Path(r"20251003/20251003/ADP_DTM_FACT.Proyeccion.csv")
ruta_items =  Path(r"20251003/20251003/ADP_DTM_DIM.Items.csv")

archivos = {
    "acta": ruta_acta,
    "insumo": ruta_insumo,
    "proyeccion": ruta_proyeccion,
    "items": ruta_items
}

# === Lectura y muestra de columnas ===
for nombre, ruta in archivos.items():
    try:
        df = pd.read_csv(ruta, nrows=5)
        print(f"/n=== {nombre.upper()} ===")
        print(f"Columnas detectadas: {list(df.columns)}")
    except Exception as e:
        print(f"/n[ERROR] No se pudo leer {nombre}: {e}")


/n=== ACTA ===
Columnas detectadas: ['SkIdEmpresa', 'SkIdProyecto', 'SkIdFecha', 'SkIdEstado', 'SkIdInsumo', 'SkIdItems', 'SkIdEspecificacionActas', 'SkIdTercero', 'Porcentaje Anticipo', 'Valor Anticipo', 'Porcentaje Retencion Antcipo', 'Valor Retencion Anticipo', 'Porcentaje Retencion Garantia', 'Valor Retencion Garantias', 'Valor Descuentos', 'Valor Total Neto', 'Valor Iva Total', 'Valor Total Acta', 'Cantidad Acta', 'Valor Unitario', 'Valor Iva Unitario', 'Valor Total', 'No Contrato', 'Tipo Acta', 'No Acta', 'Porcentaje Retencion Garantia Fic', 'Valor Garantias Fic']
/n=== INSUMO ===
Columnas detectadas: ['SkIdInsumo', 'SkIdEmpresa', 'Empresa', 'Codigo Insumo', 'Insumo Descripcion', 'Agrupacion', 'Agrupacion Descripcion', 'Tipo', 'Tipo Descripcion', 'Unidad', 'Descripcion Unidad', 'Estado', 'Requiere Equipo', 'Dias Reposicion', 'SubAnalisis', 'Devolutivo', 'Stock Maximo', 'Stock Minimo', 'Valor Unitario', 'Porcentaje IVA', 'Valor Neto', 'Fecha Creacion', 'Fecha Modificacion', 'Codig

# empiezan las consultas

## proyectos y macroproyectos


In [10]:
import pandas as pd

ruta_macro =  Path(r"20251003/20251003/proyectos_macroproyectos.csv")

df_macro = pd.read_csv(ruta_macro, nrows=5)
print("Columnas detectadas en proyectos_macroproyectos.csv:")
for c in df_macro.columns:
    print(" -", c)


Columnas detectadas en proyectos_macroproyectos.csv:
 - Proyecto
 - Sucursal
 - Centro de costo
 - Estado
 - Tipo de proyecto
 - Macroproyecto


## proyección y acta -- planeación y ejecución

In [11]:
# Celda 2 · Consultas de relación Ítems ↔ Insumos

import pandas as pd

# === Mapear nombres EXACTOS de columnas (según tu salida) ===
col_id_item = "SkIdItems"
col_id_insumo = "SkIdInsumo"
col_desc_item = "Item Descripcion"
col_desc_insumo = "Insumo Descripcion"

col_proyecto = "SkIdProyecto"
col_fecha_proy = "SkIdFecha"
col_fecha_real_proy = "SkIdFecha Real"  # en Proyeccion
col_fecha_acta = "SkIdFecha"            # en Acta

col_cantidad_proy = "Cantidad"
col_cantidad_acta = "Cantidad Acta"

# === Limpiar duplicados y quedarnos con lo esencial para relacionar ===
pares_proyeccion = (
    df_proyeccion[[col_id_item, col_id_insumo, col_proyecto]]
    .dropna(subset=[col_id_item, col_id_insumo])
    .drop_duplicates()
)

pares_acta = (
    df_acta[[col_id_item, col_id_insumo, col_proyecto]]
    .dropna(subset=[col_id_item, col_id_insumo])
    .drop_duplicates()
)

# === Añadir descripciones legibles ===
items_min = df_items[[col_id_item, col_desc_item]].drop_duplicates()
insumos_min = df_insumo[[col_id_insumo, col_desc_insumo]].drop_duplicates()

rel_proyeccion = (
    pares_proyeccion
    .merge(items_min, on=col_id_item, how="left")
    .merge(insumos_min, on=col_id_insumo, how="left")
)

rel_acta = (
    pares_acta
    .merge(items_min, on=col_id_item, how="left")
    .merge(insumos_min, on=col_id_insumo, how="left")
)

print("[DEBUG] Pares únicos Item–Insumo (proyección):", len(rel_proyeccion))
print("[DEBUG] Pares únicos Item–Insumo (acta):", len(rel_acta))

display(rel_proyeccion.head(10))
display(rel_acta.head(10))

# =========================================================
# Consultas típicas (ajusta los IDS según tu proyecto/ítem)
# =========================================================

def obtener_insumos_de_item_en_proyecto(skiditems: int, skidproyecto: int) -> pd.DataFrame:
    print(f"[DEBUG] Consultando PROYECCION → insumos de item={skiditems} en proyecto={skidproyecto}")
    df = rel_proyeccion.query(f"{col_id_item} == @skiditems and {col_proyecto} == @skidproyecto")
    return df[[col_id_item, col_desc_item, col_id_insumo, col_desc_insumo, col_proyecto]].drop_duplicates()

def obtener_insumos_de_item_en_acta(skiditems: int, skidproyecto: int) -> pd.DataFrame:
    print(f"[DEBUG] Consultando ACTA → insumos de item={skiditems} en proyecto={skidproyecto}")
    df = rel_acta.query(f"{col_id_item} == @skiditems and {col_proyecto} == @skidproyecto")
    return df[[col_id_item, col_desc_item, col_id_insumo, col_desc_insumo, col_proyecto]].drop_duplicates()

def obtener_items_que_usan_insumo_en_proyecto(skidinsumo: int, skidproyecto: int) -> pd.DataFrame:
    print(f"[DEBUG] Consultando PROYECCION → items que usan insumo={skidinsumo} en proyecto={skidproyecto}")
    df = rel_proyeccion.query(f"{col_id_insumo} == @skidinsumo and {col_proyecto} == @skidproyecto")
    return df[[col_id_item, col_desc_item, col_id_insumo, col_desc_insumo, col_proyecto]].drop_duplicates()

def obtener_items_que_usan_insumo_en_acta(skidinsumo: int, skidproyecto: int) -> pd.DataFrame:
    print(f"[DEBUG] Consultando ACTA → items que usan insumo={skidinsumo} en proyecto={skidproyecto}")
    df = rel_acta.query(f"{col_id_insumo} == @skidinsumo and {col_proyecto} == @skidproyecto")
    return df[[col_id_item, col_desc_item, col_id_insumo, col_desc_insumo, col_proyecto]].drop_duplicates()

# =========================================================
# Comparaciones planeado vs ejecutado (por proyecto)
# =========================================================

def obtener_pares_planeados_no_ejecutados(skidproyecto: int) -> pd.DataFrame:
    """
    Pares (Item, Insumo) que aparecen en PROYECCION pero no en ACTA para ese proyecto.
    """
    print(f"[DEBUG] Comparando proyección vs acta en proyecto={skidproyecto}")
    p = rel_proyeccion.query(f"{col_proyecto} == @skidproyecto")[[col_id_item, col_id_insumo]].drop_duplicates()
    a = rel_acta.query(f"{col_proyecto} == @skidproyecto")[[col_id_item, col_id_insumo]].drop_duplicates()
    diff = p.merge(a, on=[col_id_item, col_id_insumo], how="left", indicator=True)
    diff = diff[diff["_merge"] == "left_only"].drop(columns=["_merge"])
    return (
        diff
        .merge(items_min, on=col_id_item, how="left")
        .merge(insumos_min, on=col_id_insumo, how="left")
        .assign(**{col_proyecto: skidproyecto})
        [[col_id_item, col_desc_item, col_id_insumo, col_desc_insumo, col_proyecto]]
        .drop_duplicates()
    )

def obtener_pares_ejecutados_no_planeados(skidproyecto: int) -> pd.DataFrame:
    """
    Pares (Item, Insumo) que aparecen en ACTA pero no en PROYECCION para ese proyecto.
    """
    print(f"[DEBUG] Comparando acta vs proyección en proyecto={skidproyecto}")
    a = rel_acta.query(f"{col_proyecto} == @skidproyecto")[[col_id_item, col_id_insumo]].drop_duplicates()
    p = rel_proyeccion.query(f"{col_proyecto} == @skidproyecto")[[col_id_item, col_id_insumo]].drop_duplicates()
    diff = a.merge(p, on=[col_id_item, col_id_insumo], how="left", indicator=True)
    diff = diff[diff["_merge"] == "left_only"].drop(columns=["_merge"])
    return (
        diff
        .merge(items_min, on=col_id_item, how="left")
        .merge(insumos_min, on=col_id_insumo, how="left")
        .assign(**{col_proyecto: skidproyecto})
        [[col_id_item, col_desc_item, col_id_insumo, col_desc_insumo, col_proyecto]]
        .drop_duplicates()
    )

# =============== EJEMPLOS (ajusta ids a tu caso) ===============
# Usa valores reales de SkIdProyecto / SkIdItems / SkIdInsumo que existan en tus datos.

# ejemplo_proyecto = 12345
# ejemplo_item = 67890
# ejemplo_insumo = 22222

# display(obtener_insumos_de_item_en_proyecto(ejemplo_item, ejemplo_proyecto).head(20))
# display(obtener_insumos_de_item_en_acta(ejemplo_item, ejemplo_proyecto).head(20))
# display(obtener_items_que_usan_insumo_en_proyecto(ejemplo_insumo, ejemplo_proyecto).head(20))
# display(obtener_items_que_usan_insumo_en_acta(ejemplo_insumo, ejemplo_proyecto).head(20))

# display(obtener_pares_planeados_no_ejecutados(ejemplo_proyecto).head(20))
# display(obtener_pares_ejecutados_no_planeados(ejemplo_proyecto).head(20))


NameError: name 'df_proyeccion' is not defined

## items por proyecto

In [12]:
import pandas as pd

ruta_tabla_proyeccion =  Path(r"20251003/20251003/ADP_DTM_FACT.Proyeccion.csv")
ruta_tabla_items =  Path(r"20251003/20251003/ADP_DTM_DIM.Items.csv")

columnas_proyeccion = ["SkIdItems", "SkIdProyecto"]
columnas_items = ["SkIdItems", "Item No", "Item Descripcion"]

tabla_proyeccion = pd.read_csv(ruta_tabla_proyeccion, usecols=columnas_proyeccion).drop_duplicates()
tabla_items = pd.read_csv(ruta_tabla_items, usecols=columnas_items).drop_duplicates()

print(f"[DEBUG] Número de filas en tabla de proyección = {len(tabla_proyeccion)}")
print(f"[DEBUG] Número de filas en tabla de ítems = {len(tabla_items)}")
print(f"[DEBUG] Identificadores de ítems únicos en proyección = {tabla_proyeccion['SkIdItems'].nunique()}")
print(f"[DEBUG] Identificadores de ítems únicos en ítems = {tabla_items['SkIdItems'].nunique()}")

tabla_proyeccion["SkIdItems"] = tabla_proyeccion["SkIdItems"].astype(str).str.strip()
tabla_items["SkIdItems"] = tabla_items["SkIdItems"].astype(str).str.strip()

tabla_unida = tabla_proyeccion.merge(tabla_items, on="SkIdItems", how="left")

print(f"[DEBUG] Filas después del merge = {len(tabla_unida)}")
print(f"[DEBUG] Registros con descripción de ítem ausente = {tabla_unida['Item Descripcion'].isna().sum()}")

# Identificar claves que no tienen coincidencia
izquierda_sin_coincidencia = tabla_proyeccion.merge(
    tabla_items[["SkIdItems"]], on="SkIdItems", how="left", indicator=True
)
izquierda_sin_coincidencia = izquierda_sin_coincidencia[
    izquierda_sin_coincidencia["_merge"] == "left_only"
]["SkIdItems"].unique()

derecha_sin_coincidencia = tabla_items.merge(
    tabla_proyeccion[["SkIdItems"]], on="SkIdItems", how="left", indicator=True
)
derecha_sin_coincidencia = derecha_sin_coincidencia[
    derecha_sin_coincidencia["_merge"] == "left_only"
]["SkIdItems"].unique()

print(f"[DEBUG] Ítems presentes en proyección pero no en ítems: {len(izquierda_sin_coincidencia)}")
print(f"[DEBUG] Ítems presentes en ítems pero no en proyección: {len(derecha_sin_coincidencia)}")

print("/n[DEBUG] Ejemplo de filas unidas:")
display(tabla_unida.head(5))

if len(izquierda_sin_coincidencia) > 0:
    print("/n[DEBUG] Ejemplos de identificadores sin coincidencia en ítems:")
    display(pd.DataFrame({"SkIdItems": izquierda_sin_coincidencia[:5]}))

if len(derecha_sin_coincidencia) > 0:
    print("/n[DEBUG] Ejemplos de identificadores sin coincidencia en proyección:")
    display(pd.DataFrame({"SkIdItems": derecha_sin_coincidencia[:5]}))


[DEBUG] Número de filas en tabla de proyección = 33200
[DEBUG] Número de filas en tabla de ítems = 49973
[DEBUG] Identificadores de ítems únicos en proyección = 33200
[DEBUG] Identificadores de ítems únicos en ítems = 49973
[DEBUG] Filas después del merge = 33200
[DEBUG] Registros con descripción de ítem ausente = 0
[DEBUG] Ítems presentes en proyección pero no en ítems: 0
[DEBUG] Ítems presentes en ítems pero no en proyección: 16773
/n[DEBUG] Ejemplo de filas unidas:


,SkIdProyecto,SkIdItems,Item No,Item Descripcion
0,1005,1006583,2.1,Pilote preexcavado d=80cm
1,1005,1007643,2.05,Pantalla preexcavada
2,1005,1005427,2.06,Pilote tipo barrete
3,1005,1006582,2.07,Pilote preexcavado d=50cm
4,1005,1005428,2.08,Pilote preexcavado d=60cm


/n[DEBUG] Ejemplos de identificadores sin coincidencia en proyección:


,SkIdItems
0,1005213
1,1005283
2,1005355
3,1007833
4,1004993


In [13]:
import pandas as pd

# Rutas de archivos
ruta_tabla_proyeccion =  Path(r"20251003/20251003/ADP_DTM_FACT.Proyeccion.csv")
ruta_tabla_items =  Path(r"20251003/20251003/ADP_DTM_DIM.Items.csv")
ruta_tabla_proyectos =  Path(r"20251003/20251003/ADP_DTM_DIM.Proyecto.csv")

# Columnas a utilizar
columnas_proyeccion = ["SkIdProyecto", "SkIdItems"]
columnas_items = ["SkIdItems", "Item No", "Item Descripcion"]
columnas_proyectos = [
    "SkIdProyecto", "Codigo Proyecto", "Nombre Proyecto", "Clase Proyecto", "Tipo", "Estado",
    "Presupuesto Fijo", "Propietario", "Sucursal", "Sucursal Nombre", "MacroProyecto",
    "MacroProyecto Descripcion", "Centro Costo", "Centro Costo Descripcion", "VIS",
    "Sucursal Administrativa", "SkIdEmpresa", "Empresa", "Fecha De Elaboracion", "Fecha De Inicio",
    "Fecha De Finalizacion", "UnidadAConstruir", "CantidadAConstruir", "AreaAConstruir_M2",
    "AreaConstruidaFinal_M2", "AreaAVender_M2", "Arealote_M2", "CostoPreFactibilidad", "Iniciales",
    "Nocontrato", "Cliente", "Inversionista", "Almacenista", "PorcentajeAdministracion",
    "PorcentajeImprevistos", "PorcentajeUtilidad", "IVA"
]

# Carga de datos
tabla_proyeccion = pd.read_csv(ruta_tabla_proyeccion, usecols=columnas_proyeccion).drop_duplicates()
tabla_items = pd.read_csv(ruta_tabla_items, usecols=columnas_items).drop_duplicates()
tabla_proyectos = pd.read_csv(ruta_tabla_proyectos, usecols=columnas_proyectos).drop_duplicates()

print(f"[DEBUG] Filas en tabla de proyeccion = {len(tabla_proyeccion)}")
print(f"[DEBUG] Filas en tabla de items = {len(tabla_items)}")
print(f"[DEBUG] Filas en tabla de proyectos = {len(tabla_proyectos)}")

print(f"[DEBUG] Columnas en tabla de proyeccion = {list(tabla_proyeccion.columns)}")
print(f"[DEBUG] Columnas en tabla de items = {list(tabla_items.columns)}")
print(f"[DEBUG] Columnas en tabla de proyectos = {list(tabla_proyectos.columns)}")

print(f"[DEBUG] Claves unicas SkIdItems en proyeccion = {tabla_proyeccion['SkIdItems'].nunique()}")
print(f"[DEBUG] Claves unicas SkIdItems en items = {tabla_items['SkIdItems'].nunique()}")
print(f"[DEBUG] Claves unicas SkIdProyecto en proyeccion = {tabla_proyeccion['SkIdProyecto'].nunique()}")
print(f"[DEBUG] Claves unicas SkIdProyecto en proyectos = {tabla_proyectos['SkIdProyecto'].nunique()}")

# Normalizar tipos y espacios en las llaves
tabla_proyeccion["SkIdItems"] = tabla_proyeccion["SkIdItems"].astype(str).str.strip()
tabla_items["SkIdItems"] = tabla_items["SkIdItems"].astype(str).str.strip()

tabla_proyeccion["SkIdProyecto"] = tabla_proyeccion["SkIdProyecto"].astype(str).str.strip()
tabla_proyectos["SkIdProyecto"] = tabla_proyectos["SkIdProyecto"].astype(str).str.strip()
tabla_proyectos["Centro Costo"] = tabla_proyectos["Centro Costo"].astype(str).str.strip()

print(f"[DEBUG] Tipos de datos luego de normalizar:")
print(f"        proyeccion['SkIdItems'] dtype = {tabla_proyeccion['SkIdItems'].dtype}")
print(f"        items['SkIdItems'] dtype       = {tabla_items['SkIdItems'].dtype}")
print(f"        proyeccion['SkIdProyecto'] dtype = {tabla_proyeccion['SkIdProyecto'].dtype}")
print(f"        proyectos['SkIdProyecto'] dtype  = {tabla_proyectos['SkIdProyecto'].dtype}")

# Verificacion de muestras de llaves
print(f"[DEBUG] Muestra de SkIdItems en proyeccion: {tabla_proyeccion['SkIdItems'].head(5).tolist()}")
print(f"[DEBUG] Muestra de SkIdItems en items: {tabla_items['SkIdItems'].head(5).tolist()}")
print(f"[DEBUG] Muestra de SkIdProyecto en proyeccion: {tabla_proyeccion['SkIdProyecto'].head(5).tolist()}")
print(f"[DEBUG] Muestra de SkIdProyecto en proyectos: {tabla_proyectos['SkIdProyecto'].head(5).tolist()}")

# Merge entre Proyeccion e Items por SkIdItems
tabla_proyeccion_items = tabla_proyeccion.merge(tabla_items, on="SkIdItems", how="left")
print(f"[DEBUG] Filas despues de unir Proyeccion con Items = {len(tabla_proyeccion_items)}")
print(f"[DEBUG] Registros sin descripcion de item tras la union = {tabla_proyeccion_items['Item Descripcion'].isna().sum()}")

# Identificar claves de items no coincidentes
items_faltantes_en_items = (
    tabla_proyeccion.merge(tabla_items[["SkIdItems"]], on="SkIdItems", how="left", indicator=True)
)
items_faltantes_en_items = items_faltantes_en_items[items_faltantes_en_items["_merge"] == "left_only"]["SkIdItems"].unique()
print(f"[DEBUG] Cantidad de SkIdItems presentes en proyeccion sin coincidencia en items = {len(items_faltantes_en_items)}")
if len(items_faltantes_en_items) > 0:
    print(f"[DEBUG] Ejemplos de SkIdItems sin coincidencia (hasta 10): {items_faltantes_en_items[:10].tolist()}")

# Merge entre (Proyeccion+Items) y Proyectos por SkIdProyecto
tabla_proyeccion_items_proyectos = tabla_proyeccion_items.merge(
    tabla_proyectos[["SkIdProyecto", "Nombre Proyecto", "MacroProyecto", "Tipo", "Estado", "Centro Costo"]],
    on="SkIdProyecto",
    how="left"
)

print(f"[DEBUG] Filas despues de unir con Proyectos = {len(tabla_proyeccion_items_proyectos)}")
print(f"[DEBUG] Registros sin nombre de proyecto tras la union = {tabla_proyeccion_items_proyectos['Nombre Proyecto'].isna().sum()}")

# Chequeo de proyectos sin coincidencia (por si hiciera falta unir por Centro Costo en lugar de SkIdProyecto)
proyectos_sin_coincidencia = (
    tabla_proyeccion.merge(tabla_proyectos[["SkIdProyecto"]], on="SkIdProyecto", how="left", indicator=True)
)
proyectos_sin_coincidencia = proyectos_sin_coincidencia[proyectos_sin_coincidencia["_merge"] == "left_only"]["SkIdProyecto"].unique()
print(f"[DEBUG] Cantidad de SkIdProyecto de proyeccion sin coincidencia en proyectos (por SkIdProyecto) = {len(proyectos_sin_coincidencia)}")
if len(proyectos_sin_coincidencia) > 0:
    print(f"[DEBUG] Ejemplos de SkIdProyecto sin coincidencia (hasta 10): {proyectos_sin_coincidencia[:10].tolist()}")

# Si faltaran muchas coincidencias, sugerir prueba alternativa por Centro Costo:
if tabla_proyeccion_items_proyectos["Nombre Proyecto"].isna().sum() > 0:
    print("[DEBUG] Advertencia: Existen proyectos sin nombre. Si fuera necesario, intente unir por 'Centro Costo' en lugar de 'SkIdProyecto'.")

# Calculo de numero de items por nombre de proyecto
resumen_items_por_nombre_de_proyecto = (
    tabla_proyeccion_items_proyectos
    .groupby("Nombre Proyecto")
    .agg(
        cantidad_filas=("SkIdItems", "count"),
        cantidad_items_unicos=("SkIdItems", "nunique")
    )
    .sort_values(["cantidad_items_unicos", "cantidad_filas"], ascending=False)
    .reset_index()
)

print(f"[DEBUG] Numero total de proyectos con al menos un item = {len(resumen_items_por_nombre_de_proyecto)}")
print("[DEBUG] Proyectos con mayor cantidad de items unicos (vista previa):")
print(resumen_items_por_nombre_de_proyecto.head(10).to_string(index=False))

# Tabla legible de ejemplo antes del display
tabla_legible_previa = tabla_proyeccion_items_proyectos[
    ["Nombre Proyecto", "MacroProyecto", "Tipo", "Estado", "SkIdItems", "Item No", "Item Descripcion"]
].drop_duplicates()

print(f"[DEBUG] Filas en tabla legible previa = {len(tabla_legible_previa)}")
print("[DEBUG] Ejemplo de filas legibles (primeras 10):")
print(tabla_legible_previa.head(10).to_string(index=False))

# Displays finales
display(resumen_items_por_nombre_de_proyecto.head(20))
display(tabla_legible_previa.head(20))


[DEBUG] Filas en tabla de proyeccion = 33200
[DEBUG] Filas en tabla de items = 49973
[DEBUG] Filas en tabla de proyectos = 85
[DEBUG] Columnas en tabla de proyeccion = ['SkIdProyecto', 'SkIdItems']
[DEBUG] Columnas en tabla de items = ['SkIdItems', 'Item No', 'Item Descripcion']
[DEBUG] Columnas en tabla de proyectos = ['SkIdProyecto', 'Codigo Proyecto', 'Nombre Proyecto', 'Clase Proyecto', 'Tipo', 'Estado', 'Presupuesto Fijo', 'Propietario', 'Sucursal', 'Sucursal Nombre', 'MacroProyecto', 'MacroProyecto Descripcion', 'Centro Costo', 'Centro Costo Descripcion', 'VIS', 'Sucursal Administrativa', 'SkIdEmpresa', 'Empresa', 'Fecha De Elaboracion', 'Fecha De Inicio', 'Fecha De Finalizacion', 'UnidadAConstruir', 'CantidadAConstruir', 'AreaAConstruir_M2', 'AreaConstruidaFinal_M2', 'AreaAVender_M2', 'Arealote_M2', 'CostoPreFactibilidad', 'Iniciales', 'Nocontrato', 'Cliente', 'Inversionista', 'Almacenista', 'PorcentajeAdministracion', 'PorcentajeImprevistos', 'PorcentajeUtilidad', 'IVA']
[DEBUG

,Nombre Proyecto,cantidad_filas,cantidad_items_unicos
0,El Polo 1 - Etapa 1 - Torre 1,2638,2638
1,Four Seasons San Francisco,2060,2060
2,Valverde - Etapa Menta,1702,1702
3,Valverde - Palma,1538,1538
4,Los Samanes,1272,1272
5,ATRIO - Torre Norte,1108,1108
6,TRIBECA - Etapas I y II,1039,1039
7,Valverde - Etapa Olivo,996,996
8,URBAN PLAZA,830,830
9,Edificio Teleskop,699,699


,Nombre Proyecto,MacroProyecto,Tipo,Estado,SkIdItems,Item No,Item Descripcion
0,URBAN PLAZA,NaN,ADPRO,Finalizado,1006583,2.1,Pilote preexcavado d=80cm
1,URBAN PLAZA,NaN,ADPRO,Finalizado,1007643,2.05,Pantalla preexcavada
2,URBAN PLAZA,NaN,ADPRO,Finalizado,1005427,2.06,Pilote tipo barrete
3,URBAN PLAZA,NaN,ADPRO,Finalizado,1006582,2.07,Pilote preexcavado d=50cm
4,URBAN PLAZA,NaN,ADPRO,Finalizado,1005428,2.08,Pilote preexcavado d=60cm
5,URBAN PLAZA,NaN,ADPRO,Finalizado,1005429,2.09,Pilote preexcavado d=70cm
6,URBAN PLAZA,NaN,ADPRO,Finalizado,1005446,2.35,Acero de 60000 psi pilotes/pantallas
7,URBAN PLAZA,NaN,ADPRO,Finalizado,1005419,2.13,Excavación mecánica 2o sótano
8,URBAN PLAZA,NaN,ADPRO,Finalizado,1005420,2.14,Excavación mecánica 3er sótano
9,URBAN PLAZA,NaN,ADPRO,Finalizado,1006594,2.21,Placa andén h=0.60m


In [11]:
import pandas as pd

# === RUTAS ===
ruta_tabla_proyeccion =  Path(r"20251003/20251003/ADP_DTM_FACT.Proyeccion.csv")
ruta_tabla_items =  Path(r"20251003/20251003/ADP_DTM_DIM.Items.csv")
ruta_tabla_proyectos =  Path(r"20251003/20251003/ADP_DTM_DIM.Proyecto.csv")

# === CARGA ===
tabla_proyeccion = pd.read_csv(ruta_tabla_proyeccion, usecols=["SkIdProyecto", "SkIdItems"]).drop_duplicates()
tabla_items = pd.read_csv(ruta_tabla_items, usecols=["SkIdItems", "Item No", "Item Descripcion"]).drop_duplicates()
tabla_proyectos = pd.read_csv(ruta_tabla_proyectos, usecols=["SkIdProyecto", "MacroProyecto", "Nombre Proyecto"]).drop_duplicates()

# === NORMALIZACIÓN ===
tabla_proyeccion["SkIdProyecto"] = tabla_proyeccion["SkIdProyecto"].astype(str).str.strip()
tabla_proyectos["SkIdProyecto"] = tabla_proyectos["SkIdProyecto"].astype(str).str.strip()

tabla_proyeccion["SkIdItems"] = tabla_proyeccion["SkIdItems"].astype(str).str.strip()
tabla_items["SkIdItems"] = tabla_items["SkIdItems"].astype(str).str.strip()

# === MERGES ===
tabla_proyeccion_items = tabla_proyeccion.merge(tabla_items, on="SkIdItems", how="left")
tabla_total = tabla_proyeccion_items.merge(tabla_proyectos, on="SkIdProyecto", how="left")

print(f"[DEBUG] Filas tras unir todo = {len(tabla_total)}")
print(f"[DEBUG] Registros sin macroproyecto = {tabla_total['MacroProyecto'].isna().sum()}")
print(f"[DEBUG] Registros sin nombre de ítem = {tabla_total['Item Descripcion'].isna().sum()}")

# === AGRUPACIÓN POR MACROPROYECTO ===
resumen_por_macroproyecto = (
    tabla_total
    .groupby("MacroProyecto")
    .agg(
        cantidad_filas=("SkIdItems", "count"),
        cantidad_items_unicos=("SkIdItems", "nunique")
    )
    .sort_values("cantidad_items_unicos", ascending=False)
    .reset_index()
)

print(f"[DEBUG] Número total de macroproyectos = {len(resumen_por_macroproyecto)}")
print("[DEBUG] Vista previa de macroproyectos con más ítems únicos:")
print(resumen_por_macroproyecto.head(10).to_string(index=False))

# === EJEMPLOS DETALLADOS ===
tabla_ejemplo = tabla_total[
    ["MacroProyecto", "Nombre Proyecto", "Item No", "Item Descripcion"]
].drop_duplicates()

print("[DEBUG] Ejemplo de filas legibles (primeras 15):")
print(tabla_ejemplo.head(15).to_string(index=False))

display(resumen_por_macroproyecto.head(20))
display(tabla_ejemplo.head(20))


[DEBUG] Filas tras unir todo = 33200
[DEBUG] Registros sin macroproyecto = 22044
[DEBUG] Registros sin nombre de ítem = 0
[DEBUG] Número total de macroproyectos = 10
[DEBUG] Vista previa de macroproyectos con más ítems únicos:
 MacroProyecto  cantidad_filas  cantidad_items_unicos
           4.0            2424                   2424
           8.0            2309                   2309
         107.0            2241                   2241
          14.0            1329                   1329
           1.0            1157                   1157
          13.0             737                    737
           3.0             592                    592
         101.0             220                    220
         103.0              91                     91
          15.0              56                     56
[DEBUG] Ejemplo de filas legibles (primeras 15):
 MacroProyecto Nombre Proyecto Item No                     Item Descripcion
           NaN     URBAN PLAZA     2.1            Pilo

,MacroProyecto,cantidad_filas,cantidad_items_unicos
0,4.0,2424,2424
1,8.0,2309,2309
2,107.0,2241,2241
3,14.0,1329,1329
4,1.0,1157,1157
5,13.0,737,737
6,3.0,592,592
7,101.0,220,220
8,103.0,91,91
9,15.0,56,56


,MacroProyecto,Nombre Proyecto,Item No,Item Descripcion
0,NaN,URBAN PLAZA,2.1,Pilote preexcavado d=80cm
1,NaN,URBAN PLAZA,2.05,Pantalla preexcavada
2,NaN,URBAN PLAZA,2.06,Pilote tipo barrete
3,NaN,URBAN PLAZA,2.07,Pilote preexcavado d=50cm
4,NaN,URBAN PLAZA,2.08,Pilote preexcavado d=60cm
5,NaN,URBAN PLAZA,2.09,Pilote preexcavado d=70cm
6,NaN,URBAN PLAZA,2.35,Acero de 60000 psi pilotes/pantallas
7,NaN,URBAN PLAZA,2.13,Excavación mecánica 2o sótano
8,NaN,URBAN PLAZA,2.14,Excavación mecánica 3er sótano
9,NaN,URBAN PLAZA,2.21,Placa andén h=0.60m


# relacion entre tablas

### matriz de adyacencia

In [5]:
# matriz_adyacencia_bd.py
# -*- coding: utf-8 -*-
"""
Construye la matriz de adyacencia entre TODAS las tablas (CSVs) de una carpeta.
- Detecta columnas tipo SkId* y propone PK canónico por tabla:
    PK(tabla T) ≈ SkId{T} (tolerante a plural/singular y espacios)
- Crea aristas A -> B si A tiene una columna que coincide con el PK canónico de B.
- Genera:
    __edges_detectados.csv
    __adyacencia_dirigida.csv
    __adyacencia_no_dirigida.csv
"""

import os
import re
import sys
import glob
import pandas as pd
from typing import Dict, List, Set, Tuple

DEFAULT_FOLDER =  Path(r"20251003/20251003")


def debug(message: str) -> None:
    print(f"[DEBUG] {message}")

def get_csv_paths(base_folder: str) -> List[str]:
    paths = sorted(glob.glob(os.path.join(base_folder, "*.csv")))
    debug(f"CSV encontrados: {len(paths)}")
    return paths

def normalize_table_name_from_filename(csv_path: str) -> str:
    """
    Extrae el nombre de la 'tabla' desde el nombre del archivo.
    Ej.: 'ADP_DTM_FACT.Acta.csv' -> 'Acta'
         'ADP_DTM_DIM.Items.csv' -> 'Items'
    """
    base = os.path.basename(csv_path)
    name = os.path.splitext(base)[0]
    # Tomar segmento después del último punto si existe
    if "." in name:
        name = name.split(".")[-1]
    return name.strip()

def canonical_pk_for_table(table_name: str) -> List[str]:
    """
    Retorna posibles nombres de PK, ordenados por preferencia.
    'Items' -> ['SkIdItems', 'SkIdItem']
    'Insumo' -> ['SkIdInsumo', 'SkIdInsumos']
    Permite tolerar pequeñas variaciones.
    """
    t = re.sub(r"[\s_]+", "", table_name, flags=re.I)
    singular = t[:-1] if t.lower().endswith("s") else t
    plural = t if t.lower().endswith("s") else t + "s"
    candidates = [
        f"SkId{t}",
        f"SkId{singular}",
        f"SkId{plural}",
    ]
    # Unicos preservando orden
    seen = set()
    result = []
    for c in candidates:
        if c.lower() not in seen:
            result.append(c)
            seen.add(c.lower())
    return result

def read_header_columns(csv_path: str) -> List[str]:
    try:
        df = pd.read_csv(csv_path, nrows=0)
        return list(df.columns)
    except Exception as e:
        debug(f"No se pudo leer cabecera de {csv_path}: {e}")
        return []

def build_schema(base_folder: str) -> Tuple[Dict[str, List[str]], Dict[str, str]]:
    """
    Devuelve:
      - columns_by_table: {tabla -> [columnas]}
      - pk_by_table: {tabla -> pk_detectado}
    """
    columns_by_table: Dict[str, List[str]] = {}
    pk_by_table: Dict[str, str] = {}

    for csv_path in get_csv_paths(base_folder):
        table = normalize_table_name_from_filename(csv_path)
        cols = read_header_columns(csv_path)
        columns_by_table[table] = cols

        # Detectar PK canónico por presencia en columnas
        pk_candidates = canonical_pk_for_table(table)
        pk_found = None
        for c in cols:
            for cand in pk_candidates:
                if c.strip().lower() == cand.strip().lower():
                    pk_found = c
                    break
            if pk_found:
                break
        if pk_found is None:
            # fallback: primera columna que empiece por SkId{algo}
            for c in cols:
                if re.match(r"(?i)^SkId[A-Z0-9_ ]+$", c, flags=re.I):
                    pk_found = c
                    break
        if pk_found:
            pk_by_table[table] = pk_found
        else:
            pk_by_table[table] = ""  # sin PK claro

    return columns_by_table, pk_by_table

def build_edges(columns_by_table: Dict[str, List[str]], pk_by_table: Dict[str, str]) -> List[Tuple[str, str, str]]:
    """
    Crea aristas dirigidas (origen -> destino).
    Una arista A->B existe si A tiene una columna igual al PK detectado de B.
    Retorna lista de tuplas: (tabla_origen, tabla_destino, columna_en_origen)
    """
    # Índice inverso: pk_name_lower -> [tabla_dest]
    pk_to_table: Dict[str, List[str]] = {}
    for t, pk in pk_by_table.items():
        if pk:
            pk_to_table.setdefault(pk.strip().lower(), []).append(t)

    edges: List[Tuple[str, str, str]] = []
    for table_src, cols in columns_by_table.items():
        for c in cols:
            key = c.strip().lower()
            if key in pk_to_table:
                # No cuentes autorelación si la columna es su propio PK (A->A) salvo que existan varias tablas con mismo PK name
                for table_dst in pk_to_table[key]:
                    if not (table_dst == table_src and c == pk_by_table.get(table_src, "")):
                        edges.append((table_src, table_dst, c))
    return edges

def build_adjacency(tables: List[str], edges: List[Tuple[str, str, str]]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Devuelve dos matrices:
      - dirigida (A->B)
      - no dirigida (A—B) = dirigida OR traspuesta
    """
    index = tables
    directed = pd.DataFrame(0, index=index, columns=index, dtype=int)
    for src, dst, _ in edges:
        if src in directed.index and dst in directed.columns:
            directed.loc[src, dst] = 1
    undirected = directed.copy()
    undirected = ((undirected + undirected.T) > 0).astype(int)
    return directed, undirected

def save_outputs(base_folder: str, edges: List[Tuple[str, str, str]], directed: pd.DataFrame, undirected: pd.DataFrame) -> None:
    edges_df = pd.DataFrame(edges, columns=["TablaOrigen", "TablaDestino", "ColumnaEnOrigen"])
    edges_csv = os.path.join(base_folder, "__edges_detectados.csv")
    dir_csv = os.path.join(base_folder, "__adyacencia_dirigida.csv")
    undir_csv = os.path.join(base_folder, "__adyacencia_no_dirigida.csv")

    edges_df.to_csv(edges_csv, index=False, encoding="utf-8-sig")
    directed.to_csv(dir_csv, encoding="utf-8-sig")
    undirected.to_csv(undir_csv, encoding="utf-8-sig")

    print("\nListo ✅ Archivos generados:")
    print(f"• {edges_csv}")
    print(f"• {dir_csv}")
    print(f"• {undir_csv}")



In [6]:
def main() -> None:
    # 1) Carpeta objetivo
    base_folder =  Path(r"20251003/20251003")
    #if len(sys.argv) > 1 and sys.argv[1].strip():
        #base_folder = sys.argv[1].strip()
    print(f"[DEBUG] Carpeta analizada: {base_folder}")

    if not os.path.isdir(base_folder):
        print("No se encontró la carpeta. Pase la ruta como argumento o edite DEFAULT_FOLDER.")
        return

    # 2) Esquema: columnas por tabla + PKs
    columns_by_table, pk_by_table = build_schema(base_folder)
    print(f"[DEBUG] Tablas detectadas: {len(columns_by_table)}")

    # Resumen PKs
    print("\n[DEBUG] PK detectados (tabla -> pk):")
    for t, pk in sorted(pk_by_table.items()):
        print(f"  - {t}: {pk if pk else '(sin pk claro)'}")

    # 3) Aristas
    edges = build_edges(columns_by_table, pk_by_table)
    print(f"\n[DEBUG] Aristas detectadas (A->B por FK a PK): {len(edges)}")
    # Vista corta
    for row in edges[:20]:
        print("   ", row)

    # 4) Matrices de adyacencia
    tables = sorted(columns_by_table.keys())
    directed, undirected = build_adjacency(tables, edges)

    # 5) Guardar
    save_outputs(base_folder, edges, directed, undirected)

if __name__ == "__main__":
    main()


[DEBUG] Carpeta analizada: 20251003\20251003
[DEBUG] CSV encontrados: 55
[DEBUG] Tablas detectadas: 55

[DEBUG] PK detectados (tabla -> pk):
  - Acta: SkIdEmpresa
  - ActasDescuentos: SkIdEmpresa
  - Actividades: SkIdActividad
  - Anticipo: SkIdEmpresa
  - AuditoriaActas: SkIdEmpresa
  - AuditoriaContratos: SkIdEmpresa
  - AuditoriaEntradasAlmacen: SkIdEmpresa
  - AuditoriaPedidos: SkIdEmpresa
  - Bodega: SkIdBodega
  - CapituloPresupuesto: SkIdCapitulo
  - Compras: SkIdEmpresa
  - Contrato: SkIdEmpresa
  - ContratosPolizas: SkIdEmpresa
  - ControlClaseOrigen: SkIdClaseOrigen
  - ControlProyecto: SkIdEmpresa
  - Devoluciones: SkIdEmpresa
  - EjecucionCliente: SkIdEmpresa
  - EjecucionEstandar: SkIdEmpresa
  - Empresa: SkIdEmpresa
  - EntradasAlmacen: SkIdEmpresa
  - EspecicficacionDePedidos: SkIdEmpresa
  - EspecificacionDeActas: SkIdEmpresa
  - EspecificacionDeContratos: SkIdEmpresa
  - EspecificacionDeEntradasAlmacen: SkIdEmpresa
  - EspecificacionEjecucionCliente: SkIdEspecificacion

## dimensiones

In [15]:
def main() -> None:
    # 1) Carpeta objetivo
    base_folder =  Path(r"20251003/20251003")
    #if len(sys.argv) > 1 and sys.argv[1].strip():
        #base_folder = sys.argv[1].strip()
    print(f"[DEBUG] Carpeta analizada: {base_folder}")

    if not os.path.isdir(base_folder):
        print("No se encontró la carpeta. Pase la ruta como argumento o edite DEFAULT_FOLDER.")
        return

    # 2) Esquema: columnas por tabla + PKs
    columns_by_table, pk_by_table = build_schema(base_folder)
    print(f"[DEBUG] Tablas detectadas: {len(columns_by_table)}")


if __name__ == "__main__":
    main()



[DEBUG] Carpeta analizada: 20251003\20251003


NameError: name 'os' is not defined

In [9]:
# --- Add-on: list table dimensions ------------------------------------------

import csv
from pathlib import Path

def count_rows_fast(csv_path: str) -> int:
    """
    Counts data rows without loading into memory.
    Assumes the first line is a header. Returns 0 if file is empty.
    """
    try:
        with open(csv_path, "r", encoding="utf-8", newline="") as f:
            reader = csv.reader(f)
            # Count lines; subtract one for header if present
            line_count = -1  # start at -1 so that header yields 0 rows
            for _ in reader:
                line_count += 1
            return max(line_count, 0)
    except UnicodeDecodeError:
        # Fallback with latin-1 if UTF-8 fails
        with open(csv_path, "r", encoding="latin-1", newline="") as f:
            reader = csv.reader(f)
            line_count = -1
            for _ in reader:
                line_count += 1
            return max(line_count, 0)
    except Exception as e:
        debug(f"No se pudo contar filas en {csv_path}: {e}")
        return 0

def list_and_print_table_dimensions(base_folder: str) -> pd.DataFrame:
    """
    Detects and prints the dimensions (rows x columns) of each CSV 'table' in base_folder.
    Sorts by number of rows (desc), then by table name (asc).
    Saves a CSV summary '__tabla_dimensiones.csv' in the folder.
    Returns the DataFrame for programmatic use.
    """
    debug(f"Iniciando detección de dimensiones en: {base_folder}")
    csv_paths = get_csv_paths(base_folder)
    records = []

    for csv_path in csv_paths:
        table_name = normalize_table_name_from_filename(csv_path)
        header_columns = read_header_columns(csv_path)
        number_of_columns = len(header_columns)
        number_of_rows = count_rows_fast(csv_path)

        records.append({
            "Tabla": table_name,
            "ArchivoCSV": os.path.basename(csv_path),
            "Filas": number_of_rows,
            "Columnas": number_of_columns
        })

    if not records:
        print("[DEBUG] No se encontraron CSVs.")
        return pd.DataFrame(columns=["Tabla", "ArchivoCSV", "Filas", "Columnas"])

    df_dims = pd.DataFrame(records)
    df_dims.sort_values(by=["Filas", "Tabla"], ascending=[False, True], inplace=True)

    # Print neatly
    print("\n[DEBUG] Dimensiones por tabla (ordenadas por número de filas desc):")
    for _, row in df_dims.iterrows():
        print(f"  - {row['Tabla']}: {row['Filas']} x {row['Columnas']}  ({row['ArchivoCSV']})")

    # Save summary
    output_csv = os.path.join(base_folder, "__tabla_dimensiones.csv")
    df_dims.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"\nResumen guardado en: {output_csv}")

    return df_dims

# --- Optional: zero-typing entry point --------------------------------------

if __name__ == "__main__":
    # Comment out your current main() call if you only want dimensions.
    # Otherwise, you can keep both; this will run after your existing outputs.
    try:
        base_folder = str(DEFAULT_FOLDER)
    except NameError:
        base_folder = "."

    print("\n[DEBUG] Listando dimensiones de tablas...")
    list_and_print_table_dimensions(base_folder)



[DEBUG] Listando dimensiones de tablas...
[DEBUG] Iniciando detección de dimensiones en: 20251003\20251003
[DEBUG] CSV encontrados: 55

[DEBUG] Dimensiones por tabla (ordenadas por número de filas desc):
  - ControlProyecto: 1620134 x 13  (ADP_DTM_FACT.ControlProyecto.csv)
  - AuditoriaPedidos: 359786 x 8  (ADP_DTM_FACT.AuditoriaPedidos.csv)
  - InventarioResumido: 332969 x 13  (ADP_DTM_FACT.InventarioResumido.csv)
  - Proyeccion: 273450 x 19  (ADP_DTM_FACT.Proyeccion.csv)
  - Acta: 197387 x 27  (ADP_DTM_FACT.Acta.csv)
  - SalidasAlmacen: 174370 x 18  (ADP_DTM_FACT.SalidasAlmacen.csv)
  - Contrato: 141062 x 20  (ADP_DTM_FACT.Contrato.csv)
  - Compras: 127014 x 20  (ADP_DTM_FACT.Compras.csv)
  - EntradasAlmacen: 119573 x 16  (ADP_DTM_FACT.EntradasAlmacen.csv)
  - EspecicficacionDePedidos: 111067 x 5  (ADP_DTM_DIM.EspecicficacionDePedidos.csv)
  - Pedidos: 99836 x 10  (ADP_DTM_FACT.Pedidos.csv)
  - Bodega: 89647 x 4  (ADP_DTM_DIM.Bodega.csv)
  - EspecificacionDeEntradasAlmacen: 62953 x 

## missing values

In [12]:
# --- Add-on: Missing values profiler (chunked, low-memory) -------------------

import math
from pathlib import Path

def iter_csv_in_chunks(csv_path: str,
                       chunk_row_count: int = 200_000,
                       additional_na_values: List[str] = None):
    """
    Yields chunks as DataFrames without loading the full CSV into memory.
    Using dtype=str keeps parsing simple and fast; NA detection is handled by pandas.
    """
    if additional_na_values is None:
        additional_na_values = ["", " ", "NA", "N/A", "na", "n/a", "NaN", "nan", "NULL", "Null", "null"]

    try:
        for chunk in pd.read_csv(
            csv_path,
            dtype=str,                 # avoid expensive type inference
            na_filter=True,
            keep_default_na=True,
            na_values=additional_na_values,
            chunksize=chunk_row_count,
            encoding="utf-8",
            engine="c",
            low_memory=True,
        ):
            yield chunk
    except UnicodeDecodeError:
        debug(f"Reintentando con latin-1: {csv_path}")
        for chunk in pd.read_csv(
            csv_path,
            dtype=str,
            na_filter=True,
            keep_default_na=True,
            na_values=additional_na_values,
            chunksize=chunk_row_count,
            encoding="latin-1",
            engine="c",
            low_memory=True,
        ):
            yield chunk

def profile_missing_values_for_table(csv_path: str,
                                     table_name: str,
                                     chunk_row_count: int = 200_000) -> pd.DataFrame:
    """
    Returns a DataFrame with per-column missing value counts and percentages for a single table.
    Columns: Tabla, Columna, FilasTotales, Nulos, NoNulos, PorcentajeNulos
    """
    debug(f"Perfilando nulos de: {table_name}  ({os.path.basename(csv_path)})")
    total_row_count = 0
    nulls_by_column: Dict[str, int] = {}
    seen_header: List[str] = []

    for chunk in iter_csv_in_chunks(csv_path, chunk_row_count=chunk_row_count):
        if not seen_header:
            seen_header = list(chunk.columns)
            # Initialize counters on first chunk
            for column_name in seen_header:
                nulls_by_column[column_name] = 0

        # Count rows in this chunk
        rows_in_chunk = len(chunk)
        total_row_count += rows_in_chunk

        # Accumulate null counts per column
        # pandas uses NaN for missing after na_filter=True; dtype=str keeps it fast
        is_null_frame = chunk.isna()
        null_counts_series = is_null_frame.sum(axis=0)
        for column_name, null_count in null_counts_series.items():
            nulls_by_column[column_name] += int(null_count)

    if not seen_header:
        # Empty or unreadable file—return empty summary
        debug(f"Archivo sin filas o sin cabecera legible: {csv_path}")
        return pd.DataFrame(columns=["Tabla", "Columna", "FilasTotales", "Nulos", "NoNulos", "PorcentajeNulos"])

    # Build result DataFrame
    records = []
    for column_name in seen_header:
        null_count = nulls_by_column.get(column_name, 0)
        non_null_count = max(total_row_count - null_count, 0)
        percent_null = (null_count / total_row_count * 100.0) if total_row_count > 0 else 0.0
        records.append({
            "Tabla": table_name,
            "Columna": column_name,
            "FilasTotales": total_row_count,
            "Nulos": null_count,
            "NoNulos": non_null_count,
            "PorcentajeNulos": round(percent_null, 6),
        })

    result = pd.DataFrame.from_records(records)
    # Sort: highest missingness first, then by column name
    result.sort_values(by=["PorcentajeNulos", "Columna"], ascending=[False, True], inplace=True)
    return result

def profile_missing_values_for_folder(base_folder: str,
                                      output_folder_name: str = "__missing_values",
                                      chunk_row_count: int = 200_000,
                                      limit_tables_to_top_n_by_rows: int = None) -> Tuple[pd.DataFrame, List[str]]:
    """
    Profiles missing values for all CSVs in base_folder using chunked reading.
    Writes per-table CSVs and a global summary.
    - chunk_row_count: tune for memory; 200k is a good start for 300k+ row tables.
    - limit_tables_to_top_n_by_rows: if set, only processes the top N largest by rows
      (uses the previously generated __tabla_dimensiones.csv if present; otherwise scans quickly).
    Returns (global_summary_df, list_of_written_files).
    """
    debug(f"Iniciando perfil de nulos en carpeta: {base_folder}")
    csv_paths = get_csv_paths(base_folder)
    if not csv_paths:
        print("[DEBUG] No se encontraron CSVs.")
        return pd.DataFrame(), []

    # Create output folder
    output_folder_path = os.path.join(base_folder, output_folder_name)
    os.makedirs(output_folder_path, exist_ok=True)

    # If we already have dimensions file, use it to optionally restrict to top N by rows
    dimensions_csv_path = os.path.join(base_folder, "__tabla_dimensiones.csv")
    ordered_csv_paths = csv_paths
    if os.path.isfile(dimensions_csv_path):
        try:
            dims = pd.read_csv(dimensions_csv_path)
            # Expect columns: Tabla, ArchivoCSV, Filas, Columnas
            # Rebuild path from ArchivoCSV to maintain same base_folder
            dims["csv_path"] = dims["ArchivoCSV"].apply(lambda fn: os.path.join(base_folder, fn))
            dims = dims[dims["csv_path"].isin(csv_paths)]
            dims.sort_values(by=["Filas", "Tabla"], ascending=[False, True], inplace=True)
            ordered_csv_paths = dims["csv_path"].tolist()
            if limit_tables_to_top_n_by_rows is not None:
                ordered_csv_paths = ordered_csv_paths[:limit_tables_to_top_n_by_rows]
                debug(f"Procesando solo top {limit_tables_to_top_n_by_rows} por filas (según __tabla_dimensiones.csv).")
        except Exception as e:
            debug(f"No se pudo usar __tabla_dimensiones.csv: {e}")

    written_files: List[str] = []
    per_table_frames: List[pd.DataFrame] = []

    for csv_path in ordered_csv_paths:
        table_name = normalize_table_name_from_filename(csv_path)
        per_table_df = profile_missing_values_for_table(csv_path, table_name, chunk_row_count=chunk_row_count)

        # Write per-table file
        per_table_output_path = os.path.join(output_folder_path, f"missing_{table_name}.csv")
        per_table_df.to_csv(per_table_output_path, index=False, encoding="utf-8-sig")
        written_files.append(per_table_output_path)

        # For the global summary, only keep necessary columns
        per_table_frames.append(per_table_df[["Tabla", "Columna", "FilasTotales", "Nulos", "NoNulos", "PorcentajeNulos"]])

        # Brief console preview for very large tables
        preview_rows = min(5, len(per_table_df))
        print(f"\n[DEBUG] {table_name}: top {preview_rows} columnas por porcentaje de nulos")
        print(per_table_df.head(preview_rows).to_string(index=False))

    # Global summary
    if per_table_frames:
        global_summary_df = pd.concat(per_table_frames, ignore_index=True)
        # Sort by highest missingness overall, then by table, then by column
        global_summary_df.sort_values(
            by=["PorcentajeNulos", "FilasTotales", "Tabla", "Columna"],
            ascending=[False, False, True, True],
            inplace=True
        )
    else:
        global_summary_df = pd.DataFrame(columns=["Tabla", "Columna", "FilasTotales", "Nulos", "NoNulos", "PorcentajeNulos"])

    global_summary_path = os.path.join(output_folder_path, "__missing_summary_global.csv")
    global_summary_df.to_csv(global_summary_path, index=False, encoding="utf-8-sig")
    written_files.append(global_summary_path)

    print("\nListo ✅ Resúmenes de nulos generados:")
    for path in written_files:
        print(f"• {path}")

    return global_summary_df, written_files


# --- Optional: zero-typing entry point (runs after your main, or standalone) --

def run_missing_values_profiler_zero_typing() -> None:
    try:
        base_folder = str(DEFAULT_FOLDER)
    except NameError:
        base_folder = "."

    print("\n[DEBUG] Iniciando rutina de valores perdidos (bajo memoria)...")
    # Heurística: tablas muy grandes → chunks de ~200k filas
    # Puedes subir a 300k si tienes más RAM; bajar a 100k si te quedas corto.
    _summary, _files = profile_missing_values_for_folder(
        base_folder=base_folder,
        output_folder_name="__missing_values",
        chunk_row_count=200_000,
        limit_tables_to_top_n_by_rows=None  # o por ejemplo 10 para pruebas rápidas
    )

if __name__ == "__main__":
    # Mantén tu main() si quieres que corra primero lo de adyacencia.
    # Luego ejecutamos el perfil de nulos sin pedir argumentos.
    try:
        main()
    except Exception as e:
        debug(f"main() falló: {e}")

    run_missing_values_profiler_zero_typing()


[DEBUG] Carpeta analizada: 20251003\20251003
[DEBUG] CSV encontrados: 56
[DEBUG] Tablas detectadas: 56

[DEBUG] Iniciando rutina de valores perdidos (bajo memoria)...
[DEBUG] Iniciando perfil de nulos en carpeta: 20251003\20251003
[DEBUG] CSV encontrados: 56
[DEBUG] Perfilando nulos de: ControlProyecto  (ADP_DTM_FACT.ControlProyecto.csv)

[DEBUG] ControlProyecto: top 5 columnas por porcentaje de nulos
          Tabla          Columna  FilasTotales  Nulos  NoNulos  PorcentajeNulos
ControlProyecto    Valor Sin IVA       1620134 213096  1407038        13.152986
ControlProyecto        SkIdItems       1620134  13483  1606651         0.832215
ControlProyecto Origen Documento       1620134   1649  1618485         0.101782
ControlProyecto     SkIdCapitulo       1620134     38  1620096         0.002345
ControlProyecto         Cantidad       1620134     35  1620099         0.002160
[DEBUG] Perfilando nulos de: AuditoriaPedidos  (ADP_DTM_FACT.AuditoriaPedidos.csv)

[DEBUG] AuditoriaPedidos: top 5

In [24]:
# --- Add-on: frequency table of missing value percentages -------------------

def summarize_missing_value_distribution(base_folder: str,
                                          summary_csv_name: str = "__missing_values/__missing_summary_global.csv",
                                          bin_edges: List[float] = None) -> pd.DataFrame:
    """
    Reads the global missing-values summary and bins percentage ranges.
    Returns a DataFrame with frequency counts and saves it as __missing_value_bins.csv.
    """
    if bin_edges is None:
        # Define bins in percentage points (0–100)
        bin_edges = [0, 1, 5, 10, 25, 50, 75, 90, 100]

    summary_csv_path = os.path.join(base_folder, summary_csv_name)
    if not os.path.isfile(summary_csv_path):
        print(f"[DEBUG] No se encontró {summary_csv_path}")
        return pd.DataFrame(columns=["Bin", "Frecuencia"])

    df = pd.read_csv(summary_csv_path, usecols=["PorcentajeNulos"])
    df["PorcentajeNulos"] = pd.to_numeric(df["PorcentajeNulos"], errors="coerce").fillna(0.0)

    # Bin values
    df["Bin"] = pd.cut(df["PorcentajeNulos"], bins=bin_edges, right=False, include_lowest=True)
    freq = df["Bin"].value_counts().sort_index(ascending=False)

    freq_df = freq.reset_index()
    freq_df.columns = ["RangoPorcentajeNulos", "Frecuencia"]
    freq_df["PorcentajeDelTotal"] = (
        freq_df["Frecuencia"] / freq_df["Frecuencia"].sum() * 100.0
    ).round(2)

    # Save results
    output_path = os.path.join(base_folder, "__missing_values", "__missing_value_bins.csv")
    freq_df.to_csv(output_path, index=False, encoding="utf-8-sig")

    print("\n[DEBUG] Distribución de porcentajes de nulos (binned):")
    print(freq_df.to_string(index=False))
    print(f"\nResumen guardado en: {output_path}")

    return freq_df


# --- Optional: zero-typing entry point --------------------------------------

def run_missing_value_bins_zero_typing() -> None:
    try:
        base_folder = str(DEFAULT_FOLDER)
    except NameError:
        base_folder = "."

    print("\n[DEBUG] Generando tabla de frecuencias binned de porcentajes de nulos...")
    summarize_missing_value_distribution(base_folder)


In [25]:
summarize_missing_value_distribution("20251003/20251003")



[DEBUG] Distribución de porcentajes de nulos (binned):
RangoPorcentajeNulos  Frecuencia  PorcentajeDelTotal
           [90, 100)           2                0.31
            [75, 90)           3                0.46
            [50, 75)          11                1.69
            [25, 50)          10                1.53
            [10, 25)           5                0.77
             [5, 10)           4                0.61
              [1, 5)          14                2.15
              [0, 1)         603               92.48

Resumen guardado en: 20251003/20251003\__missing_values\__missing_value_bins.csv


,RangoPorcentajeNulos,Frecuencia,PorcentajeDelTotal
0,"[90, 100)",2,0.31
1,"[75, 90)",3,0.46
2,"[50, 75)",11,1.69
3,"[25, 50)",10,1.53
4,"[10, 25)",5,0.77
5,"[5, 10)",4,0.61
6,"[1, 5)",14,2.15
7,"[0, 1)",603,92.48


## full profile

In [17]:
!pip install --upgrade --force-reinstall numpy dataprofiler


'pip' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# full_profile_dataprep.py
# -*- coding: utf-8 -*-
"""
Genera un reporte HTML por cada CSV de una carpeta usando DataPrep EDA.
No requiere scipy y usa muestreo para ahorrar memoria.
"""

import os, glob
import pandas as pd
from dataprep.eda import create_report

BASE_FOLDER = r"20251003/20251003"
OUTPUT_FOLDER = os.path.join(BASE_FOLDER, "_profiles_dataprep")
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

for csv_path in glob.glob(os.path.join(BASE_FOLDER, "*.csv")):
    table_name = os.path.splitext(os.path.basename(csv_path))[0]
    print(f"[DEBUG] Generando reporte de {table_name} ...")
    try:
        df = pd.read_csv(csv_path, nrows=200_000)
        report = create_report(df, title=f"Reporte: {table_name}")
        out_path = os.path.join(OUTPUT_FOLDER, f"{table_name}.html")
        report.save(out_path)
        print(f"  ✅ Guardado: {out_path}")
    except Exception as e:
        print(f"  ⚠️ Error con {table_name}: {e}")

print("\nListo ✅ Todos los reportes disponibles en:", OUTPUT_FOLDER)
    

ModuleNotFoundError: No module named 'dataprep'

### intersecciones

In [ ]:
import pandas as pd
import numpy as np

# Rutas de archivos
ruta_tabla_proyeccion =  Path(r"20251003/20251003/ADP_DTM_FACT.Proyeccion.csv")
ruta_tabla_items      =  Path(r"20251003/20251003/ADP_DTM_DIM.Items.csv")
ruta_tabla_proyectos  =  Path(r"20251003/20251003/ADP_DTM_DIM.Proyecto.csv")

# Columnas a usar
columnas_proyeccion = ["SkIdProyecto", "SkIdItems"]
columnas_items = ["SkIdItems"]  # no necesitamos nombre aquí para intersecciones
columnas_proyectos = ["SkIdProyecto", "Nombre Proyecto"]

# Carga
tabla_proyeccion = pd.read_csv(ruta_tabla_proyeccion, usecols=columnas_proyeccion).drop_duplicates()
tabla_items = pd.read_csv(ruta_tabla_items, usecols=columnas_items).drop_duplicates()
tabla_proyectos = pd.read_csv(ruta_tabla_proyectos, usecols=columnas_proyectos).drop_duplicates()

print(f"[DEBUG] Filas proyección = {len(tabla_proyeccion)} | Ítems únicos en proyección = {tabla_proyeccion['SkIdItems'].nunique()} | Proyectos únicos = {tabla_proyeccion['SkIdProyecto'].nunique()}")

# Normalización
tabla_proyeccion["SkIdProyecto"] = tabla_proyeccion["SkIdProyecto"].astype(str).str.strip()
tabla_proyeccion["SkIdItems"]    = tabla_proyeccion["SkIdItems"].astype(str).str.strip()
tabla_proyectos["SkIdProyecto"]  = tabla_proyectos["SkIdProyecto"].astype(str).str.strip()

# Unión para tener nombres de proyecto
tabla_proyeccion_con_nombre = tabla_proyeccion.merge(tabla_proyectos, on="SkIdProyecto", how="left")
proyectos_sin_nombre = tabla_proyeccion_con_nombre["Nombre Proyecto"].isna().sum()
print(f"[DEBUG] Registros sin nombre de proyecto después de unir = {proyectos_sin_nombre}")

# Matriz de incidencia (proyecto x ítem) con presencia 0/1
print("[DEBUG] Construyendo matriz de incidencia proyecto × ítem (esto evita revisar pares manualmente)...")
matriz_incidencia = (
    tabla_proyeccion_con_nombre
    .assign(presente=1)
    .pivot_table(index="Nombre Proyecto", columns="SkIdItems", values="presente", aggfunc="max", fill_value=0)
)

print(f"[DEBUG] Tamaño matriz: proyectos={matriz_incidencia.shape[0]} × ítems={matriz_incidencia.shape[1]}")

# Intersecciones por multiplicación matricial: C = M · M^T
print("[DEBUG] Calculando intersecciones de ítems entre todos los pares de proyectos (M · M^T)...")
matriz_intersecciones = matriz_incidencia.values.dot(matriz_incidencia.values.T)
matriz_intersecciones = pd.DataFrame(
    matriz_intersecciones,
    index=matriz_incidencia.index,
    columns=matriz_incidencia.index
)

# Tamaño de cada proyecto (número de ítems únicos)
tamaño_por_proyecto = matriz_intersecciones[np.eye(matriz_intersecciones.shape[0], dtype=bool)]
tamaño_por_proyecto = pd.Series(tamaño_por_proyecto, index=matriz_intersecciones.index, name="ItemsUnicos")

print("[DEBUG] Ejemplo tamaños por proyecto (primeros 10):")
print(tamaño_por_proyecto.head(10).to_string())

# Extraer pares superiores (i<j) con conteo de intersección e índice de Jaccard
print("[DEBUG] Creando tabla de pares de proyectos con intersección y Jaccard...")
matriz_superior = np.triu(np.ones_like(matriz_intersecciones.values, dtype=bool), k=1)
pares = []
nombres = matriz_intersecciones.index.tolist()
for i in range(matriz_intersecciones.shape[0]):
    for j in range(i+1, matriz_intersecciones.shape[0]):
        interseccion = matriz_intersecciones.iat[i, j]
        if interseccion == 0:
            continue
        tamaño_i = matriz_intersecciones.iat[i, i]
        tamaño_j = matriz_intersecciones.iat[j, j]
        union = tamaño_i + tamaño_j - interseccion if (tamaño_i + tamaño_j - interseccion) > 0 else 0
        jaccard = interseccion / union if union > 0 else 0.0
        pares.append({
            "Proyecto_A": nombres[i],
            "Proyecto_B": nombres[j],
            "Interseccion_Items": int(interseccion),
            "Items_Proyecto_A": int(tamaño_i),
            "Items_Proyecto_B": int(tamaño_j),
            "Jaccard": float(jaccard)
        })

tabla_pares = pd.DataFrame(pares).sort_values(
    ["Interseccion_Items", "Jaccard"], ascending=False
).reset_index(drop=True)

print(f"[DEBUG] Número de pares con intersección positiva = {len(tabla_pares)}")
print("[DEBUG] Vista previa de los pares con mayor intersección:")
print(tabla_pares.head(15).to_string(index=False))

# Resultado visible
from IPython.display import display
display(tabla_pares.head(20))


[DEBUG] Filas proyección = 33200 | Ítems únicos en proyección = 33200 | Proyectos únicos = 76
[DEBUG] Registros sin nombre de proyecto después de unir = 0
[DEBUG] Construyendo matriz de incidencia proyecto × ítem (esto evita revisar pares manualmente)...
[DEBUG] Tamaño matriz: proyectos=76 × ítems=33200
[DEBUG] Calculando intersecciones de ítems entre todos los pares de proyectos (M · M^T)...


ValueError: Data must be 1-dimensional, got ndarray of shape (76, 76) instead

# Eda básico

### empresas y proyectos

In [ ]:
import pandas as pd

# === RUTAS ===
ruta_empresa =  Path(r"20251003/20251003/ADP_DTM_DIM.Empresa.csv")
ruta_proyecto =  Path(r"20251003/20251003/ADP_DTM_DIM.Proyecto.csv")

# === FUNCIÓN AUXILIAR ===
def mostrar_columnas(ruta, nombre):
    try:
        df = pd.read_csv(ruta, nrows=5)
        print(f"\n=== {nombre.upper()} ===")
        for c in df.columns:
            print(" -", c)
    except Exception as e:
        print(f"\n[ERROR] No se pudo leer {nombre}: {e}")

# === MOSTRAR ===
mostrar_columnas(ruta_empresa, "Empresa")
mostrar_columnas(ruta_proyecto, "Proyecto")



=== EMPRESA ===
 - SkIdEmpresa
 - NombreEmpresa
 - Nit
 - Direccion
 - Ref_IdEmpresa
 - Ref_BdConfServidor

=== PROYECTO ===
 - SkIdProyecto
 - Codigo Proyecto
 - Nombre Proyecto
 - Clase Proyecto
 - Tipo
 - Estado
 - Presupuesto Fijo
 - Propietario
 - Sucursal
 - Sucursal Nombre
 - MacroProyecto
 - MacroProyecto Descripcion
 - Centro Costo
 - Centro Costo Descripcion
 - VIS
 - Sucursal Administrativa
 - SkIdEmpresa
 - Empresa
 - Fecha De Elaboracion
 - Fecha De Inicio
 - Fecha De Finalizacion
 - UnidadAConstruir
 - CantidadAConstruir
 - AreaAConstruir_M2
 - AreaConstruidaFinal_M2
 - AreaAVender_M2
 - Arealote_M2
 - CostoPreFactibilidad
 - Iniciales
 - Nocontrato
 - Cliente
 - Inversionista
 - Almacenista
 - PorcentajeAdministracion
 - PorcentajeImprevistos
 - PorcentajeUtilidad
 - IVA


In [ ]:
import pandas as pd

# === RUTAS ===
ruta_empresa =  Path(r"20251003/20251003/ADP_DTM_DIM.Empresa.csv")
ruta_proyecto =  Path(r"20251003/20251003/ADP_DTM_DIM.Proyecto.csv")

# === CARGA ===
df_emp = pd.read_csv(ruta_empresa)
df_proj = pd.read_csv(ruta_proyecto)

print("=== 🏢 EMPRESA ===")
print(f"Total de empresas: {df_emp['SkIdEmpresa'].nunique()}")
print(f"Nombres únicos: {df_emp['NombreEmpresa'].nunique()}")
print("\nColumnas con nulos:")
print(df_emp.isna().sum()[df_emp.isna().sum() > 0])
print("\nEjemplo:")
display(df_emp.head(3))

print("\n=== 🏗️ PROYECTO ===")
print(f"Total de proyectos: {df_proj['SkIdProyecto'].nunique()}")
print(f"Estados disponibles:\n{df_proj['Estado'].value_counts(dropna=False).head()}")
print(f"\nTipos de proyecto:\n{df_proj['Tipo'].value_counts(dropna=False).head()}")
print(f"\nClases de proyecto:\n{df_proj['Clase Proyecto'].value_counts(dropna=False).head()}")
print(f"\nFechas (inicio–fin): {df_proj['Fecha De Inicio'].min()} → {df_proj['Fecha De Finalizacion'].max()}")

print("\nColumnas con nulos:")
print(df_proj.isna().sum()[df_proj.isna().sum() > 0].sort_values(ascending=False).head(10))

print("\nDistribución por empresa:")
print(df_proj['Empresa'].value_counts().head())

print("\nEjemplo:")
display(df_proj[['Codigo Proyecto','Nombre Proyecto','Estado','Tipo','MacroProyecto','AreaAConstruir_M2','Presupuesto Fijo']].head(5))


=== 🏢 EMPRESA ===
Total de empresas: 1
Nombres únicos: 1

Columnas con nulos:
Series([], dtype: int64)

Ejemplo:


,SkIdEmpresa,NombreEmpresa,Nit,Direccion,Ref_IdEmpresa,Ref_BdConfServidor
0,100,ARPRO ARQUITECTOS INGENIEROS S.A.S,860067697,CRA 19 No 90-10,1,1



=== 🏗️ PROYECTO ===
Total de proyectos: 85
Estados disponibles:
Estado
Finalizado      43
En ejecucion    29
Presupuesto     13
Name: count, dtype: int64

Tipos de proyecto:
Tipo
ADPRO        82
ADPRO-CBR     3
Name: count, dtype: int64

Clases de proyecto:
Clase Proyecto
Admon Delegada sin Representacion    43
Admon Delegada con Representacion    32
Obras a Todo Costo                    7
SIN ASIGNAR CLASE PROYECTO            3
Name: count, dtype: int64

Fechas (inicio–fin): 01/01/2021 → 31/12/2027

Columnas con nulos:
Iniciales                    85
Nocontrato                   85
Inversionista                49
Almacenista                  48
Cliente                      44
MacroProyecto                43
MacroProyecto Descripcion    43
UnidadAConstruir             38
CostoPreFactibilidad          2
AreaConstruidaFinal_M2        1
dtype: int64

Distribución por empresa:
Empresa
ARPRO ARQUITECTOS INGENIEROS S.A.S    85
Name: count, dtype: int64

Ejemplo:


,Codigo Proyecto,Nombre Proyecto,Estado,Tipo,MacroProyecto,AreaAConstruir_M2,Presupuesto Fijo
0,3,Edificio Naia,Finalizado,ADPRO,NaN,8013.46,SI
1,5,URBAN PLAZA,Finalizado,ADPRO,NaN,18000.00,SI
2,6,Parque Engativá - Etapa I,Finalizado,ADPRO,NaN,10995.00,SI
3,9,TRIBECA - Etapas I y II,Finalizado,ADPRO,NaN,1.00,SI
4,11,BS Rosales,Finalizado,ADPRO,NaN,1.00,SI


# Join proyección, insumos, items, proyectos

In [22]:
# === Rutas fijas (hardcodeadas) ===
ruta_acta =  Path(r"20251003/20251003/ADP_DTM_FACT.Acta.csv")
ruta_insumo =  Path(r"20251003/20251003/ADP_DTM_DIM.Insumo.csv")
ruta_proyectos =  Path(r"20251003/20251003/ADP_DTM_DIM.Proyecto.csv")
ruta_capitulos = Path(r"20251003/20251003/ADP_DTM_DIM.CapituloPresupuesto.csv")
ruta_proyeccion =  Path(r"20251003/20251003/ADP_DTM_FACT.Proyeccion.csv")
ruta_items =  Path(r"20251003/20251003/ADP_DTM_DIM.Items.csv")

In [23]:
# Join proyección, insumos, items, proyectos
# === CARGA ===
tabla_proyeccion = pd.read_csv(ruta_proyeccion)
tabla_items = pd.read_csv(ruta_items)
tabla_proyectos = pd.read_csv(ruta_proyectos)
tabla_capitulos = pd.read_csv(ruta_capitulos)
tabla_insumos = pd.read_csv(ruta_insumo)

C:\Users\aleja\AppData\Local\Temp\ipykernel_17624\2874475724.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  tabla_items = pd.read_csv(ruta_items)


In [24]:
tabla_base = tabla_proyeccion.copy()

# Merges previos
tabla_1 = pd.merge(tabla_base, tabla_proyectos, on="SkIdProyecto", how="left")
tabla_2 = pd.merge(tabla_1, tabla_capitulos, on="SkIdCapitulo", how="left")

# Asegura unicidad en Items
tabla_items_unica = tabla_items.drop_duplicates(subset=["SkIdItems"], keep="first")
tabla_3 = pd.merge(
    tabla_2, tabla_items_unica,
    on="SkIdItems", how="left",
    suffixes=("", "_item")   # control de sufijos para items
)

# 🔧 FIX insumos: prefijar y garantizar clave única
tabla_insumos_unica = tabla_insumos.drop_duplicates(subset=["SkIdInsumo"], keep="first").copy()
cols_no_clave_insumo = [c for c in tabla_insumos_unica.columns if c != "SkIdInsumo"]
tabla_insumos_pref = tabla_insumos_unica.rename(
    columns={c: f"Insumo_{c}" for c in cols_no_clave_insumo}
)

# (Opcional) Debug rápido de posibles overlaps antes del merge final
overlap_prev = set(tabla_3.columns).intersection(set(tabla_insumos_pref.columns))
print(f"[DEBUG] Overlap antes de merge con insumos (debería ser solo 'SkIdInsumo'): {overlap_prev}")

# Merge final (sin sufijos porque ya prefijamos)
tabla_4 = pd.merge(
    tabla_3, tabla_insumos_pref,
    on="SkIdInsumo", how="left",
    validate="many_to_one"
)

print(f"[DEBUG] OK. Forma final: {tabla_4.shape}")

tabla_4.to_csv(
    "tabla_looker.csv",
    index=False,
    encoding="utf-8",
    sep=","                
)

[DEBUG] Overlap antes de merge con insumos (debería ser solo 'SkIdInsumo'): {'SkIdInsumo'}
[DEBUG] OK. Forma final: (273450, 107)


In [26]:
# === SELECCIÓN DE COLUMNAS ===
columnas_finales = [
    "SkIdProyecto", "SkIdCapitulo", "SkIdItems", "SkIdInsumo",
    "Nombre Proyecto", "Capitulo Descripcion", "Item Descripcion", "Insumo_Insumo Descripcion", "Insumo_Agrupacion Descripcion"
    "SkIdFecha Real", "Cantidad", "Valor Unitario", "Valor Total", "Insumo_Valor Unitario", "Insumo_Valor Neto", "Insumo_Fecha Creacion",
    "Cantidad Item", "Macroproyecto Descripcion", "Insumo_Fecha Modificacion",
    "Fecha De Elaboracion", "Fecha De Inicio", "Fecha De Finalización", "SkIdFecha", "Capitulo Numero", "Cantidad_Item"
]

# Filtrar solo las columnas que existan realmente (por seguridad)
columnas_existentes = [col for col in columnas_finales if col in tabla_4.columns]
tabla_looker = tabla_4[columnas_existentes].copy()